<h1>Autoencoders

<h2>Importing required libraries

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import tensorflow as tf
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from keras.datasets import fashion_mnist
from torch.optim.lr_scheduler import ExponentialLR

<h2>Initializing device details and Importing the dataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
(xTrain, yTrain), (xTest, yTest) = fashion_mnist.load_data()

<h2>Dataset Preprocessing

In [ ]:
length, breadth = xTrain[0].shape
flattenDim = length * breadth
#xTrain = torch.FloatTensor(np.round(xTrain / 27).astype(int)).to(device)
xTrain = np.round(xTrain / 27).astype(int)
xTest = np.round(xTest / 27).astype(int)
xTrainFlat = torch.FloatTensor(np.reshape(xTrain, (-1,flattenDim))).to(device)
xTestFlat = np.reshape(xTest, (-1,flattenDim)) 

<h2>Displaying sample images from dataset

In [ ]:
def displayImages(imageList, rows, columns):
    fig, grid = plt.subplots(rows,columns) 
    for i in range(rows):
        for j in range(columns):
            grid[i,j].axis('off')
            grid[i,j].imshow(np.reshape(imageList[(i-1)*rows+j], (28,28)))

numSamples = 8
randomIndices = random.sample(range(0, len(xTrain)), numSamples)
displayImages(list(np.array(xTrain)[randomIndices]), 2, 4)

<h2>Initializing TrainLoader and TestLoader functions

In [ ]:
trainLoader = DataLoader(dataset = xTrainFlat, batch_size = 32, shuffle = True)
testLoader = DataLoader(dataset = xTestFlat, batch_size = 16, shuffle = True)

<h2>Defining AutoEncoder class

In [ ]:

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.Encoder = nn.Sequential(
        nn.Linear(in_features = 784, out_features = 512),
        nn.ReLU(),
        nn.Linear(in_features = 512, out_features = 256),
        nn.ReLU(),
        nn.Linear(in_features = 256, out_features = 128),
        nn.ReLU(),
        nn.Linear(in_features = 128, out_features = 64),
        nn.ReLU()
        )
        self.Decoder = nn.Sequential(
        nn.Linear(in_features = 64, out_features = 128),
        nn.ReLU(),
        nn.Linear(in_features = 128, out_features = 256),
        nn.ReLU(),
        nn.Linear(in_features = 256, out_features = 512),
        nn.ReLU(),
        nn.Linear(in_features = 512, out_features = 784),
        nn.ReLU()
        )
        
    def forward(self, x):
        latentRepresentation = self.Encoder(x)
        decoderOutput = self.Decoder(latentRepresentation)
        return latentRepresentation, decoderOutput

<h2>Training the AutoEncoder model

In [ ]:
aeModel = autoencoder().to(device)
modelParameters = list(aeModel.parameters())
criterion = nn.MSELoss(reduction = 'mean')
optimizer = optim.SGD(modelParameters, lr = (1e-2), momentum = 0.7)

epochLoss = []
epochList = np.arange(0, 100, dtype = int)

encoderOutputs = []
decoderOutputs = []
for epoch in range(100):
  losses = []
  for batchIndex, batchImage in enumerate(trainLoader):
    batchImage = batchImage.to(device)
    batchImage = batchImage.unsqueeze(1)
    batchLatent, reconstructedImage = aeModel(batchImage)
    loss = criterion(reconstructedImage, batchImage)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.cpu().detach().numpy())
  print("Epoch: ",epoch,"| Average loss: ",np.round(np.average(losses), 3),"| Lowest Loss: ",np.round(np.amin(losses), 3))
  epochLoss.append(np.round(np.amin(losses), 3))
  torch.save(aeModel.state_dict(), "AE.pth")

<h2>Plotting graph of loss versus epochs

In [ ]:
plt.plot(epochList, epochLoss, color = 'blue')
plt.xlabel('Epoch')
plt.ylabel('Lowest loss per epoch')
plt.legend()
plt.show()

<h2>Evaluating reconstruction capabilities on Test Set

In [ ]:
numSamples = 5
randomIndices = random.sample(range(0, len(xTest)), numSamples)
testSamples = list(np.array(xTest)[randomIndices])
latentOutputs = []
reconstructedOutputs = []
displayList = []

aeModel.eval()
for testImage in testSamples:
    encodedOutput, decodedOutput = aeModel(torch.FloatTensor(np.reshape(testImage, (-1,flattenDim))).to(device))
    latentOutputs.append(encodedOutput.cpu().detach().numpy().reshape(8, 8))
    reconstructedOutputs.append(decodedOutput.cpu().detach().numpy().reshape(28, 28))
    displayList.append(testImage)
    displayList.append(decodedOutput.cpu().detach().numpy().reshape(28, 28))

displayList = testSamples + reconstructedOutputs

displayImages(displayList, 2, 5)